## Game pattern and bye week constraints

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np


#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
#set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)


In [2]:
week = 13

In [3]:
# Create an empty model
m= gp.Model(name="NFL2b")
# ADD DECISION VARIABLES HERE
x = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")

Using license file C:\Users\Q_Ali\gurobi.lic
Academic license - for non-commercial use only


In [4]:
# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x[i,j,k]+x[j,i,k] for k in W) for i in T) == 12 for j in T)
# add constraints that Each team would play once per week
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(0,8))
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(9,13))
# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x[i,j,k]+x[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))

# add gaming pattern constraints for there are no games on week9
m.addConstrs(x[i,j,8] == 0 for i in T for j in T)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [6]:
# Add constraint for each team has a maximum distance to travel, set the maximum distance = 11000
m.addConstrs((sum(x[i,j,k] * E.iloc[i,j] * 2 for k in W for i in T) <= 11000 for j in T))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [7]:
# ADD OBJECTIVE FUNCTION
# Objective is to minimize the total travel distance of all teams
obj = sum(E.iloc[i,j]*x[i,j,k] for i in T for j in T for k in W)*2

In [8]:
#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 24504 rows, 7488 columns and 118558 nonzeros
Model fingerprint: 0xde87297c
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 23484 rows and 864 columns
Presolve time: 0.18s
Presolved: 1020 rows, 6624 columns, 59592 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 8724 iterations, 1.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  305          - 165908.000      -     -    3s
     0     0 165908.000    0  499          - 165908.000      -     -    4s
     0     0 165908.000    0  291          - 165908.000      -     -    7s
     0   

In [9]:
# ADD PRINTING
# Print optimal value of the objective function
print('\nValue of objective function: %g' % m.objVal)



Value of objective function: 165908


In [10]:
# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    if v.x !=0:
        print(f'{v.varName}={v.x}, {v.obj}')


Decision variables:
X[0,5,9]=1.0, 2226.0
X[0,6,5]=1.0, 940.0
X[0,10,3]=1.0, 1292.0
X[0,15,11]=1.0, 1586.0
X[0,18,6]=1.0, 1300.0
X[0,22,1]=1.0, 500.0
X[1,0,7]=1.0, 490.0
X[1,6,1]=1.0, 1430.0
X[1,7,0]=1.0, 1250.0
X[1,8,10]=1.0, 1074.0
X[1,11,12]=1.0, 874.0
X[1,20,4]=1.0, 1250.0
X[2,3,7]=1.0, 568.0
X[2,4,10]=1.0, 414.0
X[2,12,2]=1.0, 1056.0
X[2,13,1]=1.0, 594.0
X[2,14,4]=1.0, 688.0
X[2,23,11]=1.0, 1934.0
X[3,8,0]=1.0, 1166.0
X[3,13,2]=1.0, 526.0
X[3,14,9]=1.0, 340.0
X[3,16,6]=1.0, 574.0
X[3,19,4]=1.0, 1412.0
X[3,21,12]=1.0, 572.0
X[4,3,11]=1.0, 978.0
X[4,5,6]=1.0, 558.0
X[4,13,4]=1.0, 1014.0
X[4,16,9]=1.0, 788.0
X[4,19,0]=1.0, 2378.0
X[4,23,1]=1.0, 2346.0
X[5,2,0]=1.0, 818.0
X[5,3,3]=1.0, 1382.0
X[5,12,12]=1.0, 1870.0
X[5,13,7]=1.0, 1408.0
X[5,15,10]=1.0, 2352.0
X[5,16,4]=1.0, 1182.0
X[6,2,6]=1.0, 1850.0
X[6,4,2]=1.0, 2260.0
X[6,5,11]=1.0, 2398.0
X[6,15,0]=1.0, 696.0
X[6,18,4]=1.0, 1702.0
X[6,23,9]=1.0, 1010.0
X[7,3,1]=1.0, 1226.0
X[7,8,4]=1.0, 192.0
X[7,9,11]=1.0, 2262.0
X[7,11,3]=1.0, 

In [11]:
# print the optimal schedule for Cleveland Browns
#14:"Cleveland Browns"
rows=[]
for v in m.getVars():
    if v.varName.find("14")!= -1 and v.x==1:
        rows.append([v.varName, v.x])
print('\n Schedule for Cleveland Browns')
df = pd.DataFrame(rows, columns=["v.varName", "v.x"])
print(df)


# Q3: print distance that each team travels
distance_eachteam = {}

for a in Team_list:
    distance_travelled = 0
    for w in range(12):
        for h in range(24):
            distance_travelled += x[h,a,w].x*E.iloc[h,a]*2
        distance_eachteam[Team_list[a]] = distance_travelled

print(distance_eachteam)

keymax = max(distance_eachteam, key=distance_eachteam.get)
keymin = min(distance_eachteam, key=distance_eachteam.get)
print('\nTeam which travels the max distance:' +' '+ keymax)
print('\nTeam which travels the min distance:' +' '+  keymin)

Difference = distance_eachteam[max(distance_eachteam, key=distance_eachteam.get)]-distance_eachteam[min(distance_eachteam, key=distance_eachteam.get)]
print('\nDifference in distance travelled: %g' % Difference)



 Schedule for Cleveland Browns
      v.varName  v.x
0     X[2,14,4]  1.0
1     X[3,14,9]  1.0
2    X[12,14,0]  1.0
3    X[13,14,6]  1.0
4     X[14,4,7]  1.0
5     X[14,5,1]  1.0
6     X[14,7,5]  1.0
7   X[14,11,10]  1.0
8   X[14,16,11]  1.0
9    X[14,21,3]  1.0
10  X[19,14,12]  1.0
11   X[20,14,2]  1.0
{'Atlanta Falcons': 4602.0, 'Carolina Panthers': 7898.0, 'Chicago Bears': 6120.0, 'Detroit Lions': 5920.0, 'Green Bay Packers': 6608.0, 'Minnesota Vikings': 10324.0, 'New Orleans Saints': 6164.0, 'New York Giants': 5872.0, 'Philadelpia Eagles': 5716.0, 'Tampa Bay Buccaneers': 8510.0, 'Washington Football Team': 2908.0, 'Baltimore Ravens': 4286.0, 'Buffalo Bills': 4102.0, 'Cincinnati Bengals': 5446.0, 'Cleveland Browns': 2818.0, 'Houston Texans': 10974.0, 'Indianapolis Colts': 5692.0, 'Jacksonville Jaguars': 8520.0, 'Miami Dolphins': 10230.0, 'New England Patroits': 6064.0, 'New York Jets': 4296.0, 'Pittsburgh Steelers': 3016.0, 'Tennessee Titans': 5698.0, 'Dalls Cowboys': 7974.0}

Team 